In [1]:
import os
import time
import whitebox
import rasterio
from rasterio.crs import CRS

In [2]:
start = time.perf_counter()

In [3]:
wbt = whitebox.WhiteboxTools()

In [4]:
wbt.set_verbose_mode(False)

0

In [5]:
data_dir_rel= '../data/'
data_dir_abs = os.path.abspath(data_dir_rel)
wbt.set_working_dir(data_dir_abs)


In [6]:
laz_file = 'USGS_LPC_WY_SouthCentral_2020_D20_13TDF670640.laz'

In [7]:
raw_dem = os.path.splitext(laz_file)[0] + '_RawDEM.tif'

# Only include LAST returns, class 2 points (ground)
# 1 meter resolution
wbt.lidar_idw_interpolation(
    i=laz_file,
    output=raw_dem,
    parameter="elevation",
    returns="last",
    exclude_cls='0,1,3-18',
    resolution=1.0,
    weight=1.0,
    radius=2.5
)

0

In [8]:
# EPSG:6342 (NAD83(2011) / UTM zone 13N)
crs = CRS.from_epsg(6342)

with rasterio.open(os.path.join(data_dir_rel, raw_dem), 'r+') as dem:
    dem.crs = crs

In [9]:
filled_dem = os.path.splitext(laz_file)[0] + '_FilledDEM.tif'

# Fill NoData holes in the DEM 
wbt.fill_missing_data(
    i=raw_dem,
    output=filled_dem,
    filter=20
)

0

In [10]:
slope_rast = os.path.splitext(laz_file)[0] + '_Slope.tif'

# Calculate Slope using filled DEM
wbt.slope(
    dem=filled_dem,
    output=slope_rast,
    units='degrees'
)

0

In [11]:
aspect_rast = os.path.splitext(laz_file)[0] + '_Aspect.tif'

# Calculate Aspect using filled DEM
wbt.aspect(
    dem=filled_dem,
    output=aspect_rast
)

0

In [12]:
print(f'RUN TIME: {round(time.perf_counter() - start)} seconds')

RUN TIME: 14 seconds
